<a href="https://colab.research.google.com/github/sahil9022-crypto/Promt-engg-pro-by-sahil-pawar/blob/main/college_helper_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread pandas gspread_dataframe


In [ ]:
!pip install gspread gspread_dataframe oauth2client pandas


In [9]:
import gspread
from google.colab import auth
import google.auth
from gspread_dataframe import get_as_dataframe
import pandas as pd

# Authenticate and get credentials
auth.authenticate_user()
credentials, project_id = google.auth.default()
gc = gspread.authorize(credentials)

In [10]:
# Paste your Google Sheet URL here
sheet_url = "https://docs.google.com/spreadsheets/d/1tSRZgtigOWchbHw0kLEt8l31FqPRmnFdw60j3azPimo/edit#gid=0"

# Open the sheet and read the first worksheet
worksheet = gc.open_by_url(sheet_url).sheet1

# Convert to pandas dataframe
data = get_as_dataframe(worksheet)

# Drop blank rows
data = data.dropna(subset=["Question", "Answer"])

# Show the first 5 entries
data.head()


,Question,Answer
0,What is your company about?,"We offer GenAI, Data Science, and Cybersecurit..."
1,What is your support email?,publichacker9999@gmail.com
2,What is your refund policy?,You can request a full refund within 14 days o...
3,Do you provide internship certificates?\t,"Yes, we provide verified certificates after co..."
4,What technologies do you teach?,"We cover Python, Machine Learning, Prompt Engi..."


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [12]:
# Get the 'Question' column from your data
questions = data['Question'].fillna("").tolist()

# Initialize the vectorizer
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(questions)


In [13]:
def get_bot_answer(user_input):
    # Convert user input to the same vector space
    user_vec = vectorizer.transform([user_input])

    # Compute cosine similarity with all sheet questions
    similarity_scores = cosine_similarity(user_vec, question_vectors)

    # Get index of the best matching question
    best_match_index = similarity_scores.argmax()

    # Get the best answer
    best_question = data['Question'].iloc[best_match_index]
    best_answer = data['Answer'].iloc[best_match_index]

    # Show both matched question and answer (for clarity)
    return f"📌 Matched Question: {best_question}\n✅ Answer: {best_answer}"


In [15]:
print(get_bot_answer("What is your company about?"))


📌 Matched Question: What is your company about?
✅ Answer: We offer GenAI, Data Science, and Cybersecurity training services.
How can I enroll in a course?	You can sign up on our website or contact us via email.


In [1]:
!pip install gradio --quiet

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

# --- STEP 1: Connect to Google Sheet ---
sheet_url = 'https://docs.google.com/spreadsheets/d/1tSRZgtigOWchbHw0kLEt8l31FqPRmnFdw60j3azPimo/edit?usp=drivesdk'
csv_url = sheet_url.replace("/edit?usp=drivesdk", "/export?format=csv")
df = pd.read_csv(csv_url)

# --- STEP 2: Preprocess Questions ---
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(df['Question'])

# --- STEP 3: Define Bot Response Function ---
def get_bot_answer(user_input):
    input_vec = vectorizer.transform([user_input])
    similarity = cosine_similarity(input_vec, question_vectors)
    index = similarity.argmax()
    score = similarity[0][index]
    if score > 0.3:
        return df.iloc[index]['Answer']
    else:
        return "Sorry, I couldn't find a good answer. Please try rephrasing!"

# --- STEP 4: Gradio UI ---
iface = gr.Interface(
    fn=get_bot_answer,
    inputs="text",
    outputs="text",
    title="📘 College Study Bot",
    description="Ask any question from the syllabus. Powered by AI + your Google Sheet!"
)

iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e04edb6df97180ff9e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
import gradio as gr
import pandas as pd
import gspread
from google.colab import auth
import google.auth
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Authenticate and connect to Google Sheets
auth.authenticate_user()
credentials, project_id = google.auth.default()
gc = gspread.authorize(credentials)

sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1tSRZgtigOWchbHw0kLEt8l31FqPRmnFdw60j3azPimo/edit?usp=drivesdk')
worksheet = sheet.sheet1
data = worksheet.get_all_records()
df = pd.DataFrame(data)

# Bot logic
def chatbot_response(user_input):
    # --- STEP 2: Preprocess Questions ---
    vectorizer = TfidfVectorizer()
    question_vectors = vectorizer.fit_transform(df['Question'])

    # --- STEP 3: Define Bot Response Function ---
    input_vec = vectorizer.transform([user_input])
    similarity = cosine_similarity(input_vec, question_vectors)
    index = similarity.argmax()
    score = similarity[0][index]

    # Assuming the column names are 'Question' and 'Answer'
    if score > 0.3:
        return df.iloc[index]['Answer']
    else:
        return "Sorry, I couldn't find a good answer. Please try rephrasing!"

# Gradio UI
iface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask your question..."),
    outputs="text",
    title="College Study Helper Bot",
    description="Ask questions related to your studies"
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1998cd1e3345db2d16.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
!zip -r chatbot_project.zip /content


🧑‍💻 Built By
Sahil Pawar
Prompt Engineer | Data Science Enthusiast | Ethical Hacker
🌍 From Sangli, Maharashtra, India

📌 Future Scope
🌍 Multilingual support

🗣️ Voice input/output (speech recognition)

📲 Mobile-friendly version

💬 GPT integration for fallback answers

# 🤖 AI-Powered Study Assistant Chatbot

A smart, responsive chatbot built using **Gradio** + **Google Sheets** + **Python**, designed to help students interactively learn from a structured knowledge base. Simple to use, easy to update — just modify the Google Sheet and you're done!

---

## 🚀 Features

- ✅ Gradio chatbot UI
- ✅ Smart fuzzy logic to match closest question
- ✅ Reads from Google Sheet knowledge base
- ✅ Beginner-friendly (Google Colab compatible)
- ✅ Portable and customizable

---

## 🎯 Tech Stack

- Python 🐍
- Gradio 🎛️
- GSpread + Google Sheets API 🧾
- Google Colab (for execution)
- Markdown + GitHub for documentation

---

## 🧠 How It Works

1. Connects to a **Google Sheet** using Google credentials.
2. Accepts user input through **Gradio UI**.
3. Uses `difflib.get_close_matches()` to match user's question.
4. Displays the most relevant answer from the sheet.

---

---

## 📋 Google Sheet Format

Your Google Sheet should look like this:

| question                  | answer                           |
|--------------------------|----------------------------------|
| What is Python?          | Python is a programming language.|
| What is a loop?          | Loop is a control structure.     |

---

## 🧪 Run the Project (Google Colab or Locally)

### 1. Install Dependencies
```bash
pip install gradio gspread google-auth


## 🗂️ Folder Structure

AI-Study-Assistant/
│
├── app.py                # Main chatbot app (Gradio UI + Google Sheet logic)
├── requirements.txt      # Python dependencies
├── README.md             # Complete project documentation
└── assets/
    └── screenshot.png    # Chatbot UI screenshot (for LinkedIn/GitHub)
